In [9]:
# Importamos librerías
import json             # Módulo de codificador y decodificador JSON
import ast              # Módulo de Árboles de Sintaxis Abstracta
import pandas as pd     # Librería para manipular datasets
import pyarrow as pa    # Útil para operaciones de lectura y escritura de datos
import pyarrow.parquet as pq   # Útil para leer y escribir datos en formato Parquet de manera eficiente
import os               # creación de directorios y comprobación de existencia
from textblob import TextBlob
import unicodedata

In [15]:
# Ruta al archivo JSON
file_path = 'tip.json'

# Lista para almacenar los datos
data = []

# Abrir el archivo JSON y cargar su contenido en un objeto Python
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Cargar la línea como un objeto JSON y agregarlo a la lista
        data.append(json.loads(line))

# Si deseas visualizar los primeros elementos del archivo JSON, puedes convertirlo a un DataFrame de Pandas
df = pd.DataFrame(data)
df
# Visualizar los primeros elementos del DataFrame
#print(df.head())



,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11 19:18:57,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           908915 non-null  object
 1   business_id       908915 non-null  object
 2   text              908915 non-null  object
 3   date              908915 non-null  object
 4   compliment_count  908915 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 34.7+ MB


In [17]:
#Se creo un diccionario con claves columna y tipo
# columna - almacena los nombres de las columnas
# tipo - almacena tipos de datos únicos para cada columna
data_type_process = {
    "columna": df.columns.tolist(), 
    "tipo": [df[columna].apply(type).unique() for columna in df.columns]
}

#Se visualizara el nombre de la columna y su tipo de dato
data_type = pd.DataFrame(data_type_process)
data_type

,columna,tipo
0,user_id,[<class 'str'>]
1,business_id,[<class 'str'>]
2,text,[<class 'str'>]
3,date,[<class 'str'>]
4,compliment_count,[<class 'int'>]


In [18]:
#se utiliza la variable duplicados para guardar la busqueda y poder comparar
duplicados= df.loc[df.duplicated()]
duplicados

,user_id,business_id,text,date,compliment_count
29901,00Cz_vdInMHpTRjqbWjK5Q,ncacMQ9n_dSM1cR3c1vTQw,Miss saigon,2010-12-07 01:59:12,0
58887,47zMh_WgunwRDf7Cx2WyYw,J8vz_zwZaxzA585lV_k_vA,Com chien and mi are delish!,2011-07-23 17:22:58,0
99407,WQ8shYm0ghNDz97BuHI1fA,Sv1MEZP-mMfp8SmE0hwYEA,love the crispy buns!,2017-10-18 23:29:03,0
126358,hGxVvXVg7IK4J2aTiGMSIQ,5RsVAkDnMrcSbErS6P1eew,"Pricey, often crowded, and staff are not alway...",2017-10-20 22:08:25,0
129213,1gDoko0TrN0lnQlMC2JtXw,06PmqoU3uY5Vb-BEaMPVfw,"We ordered a pizza on New Year's Day, it never...",2014-02-20 04:09:37,0
...,...,...,...,...,...
793788,nnWFrvhO2jeq0KbG_CKCYA,jmG_QxXXfz2cnw9dQCXJLQ,"Great place, great baked goods!",2019-06-08 17:06:50,0
794036,ajDPsSD77sxcVttPfftBRQ,DTxB12bQaZ1m-nLeXWX2rw,Good service and delicious food,2017-09-12 23:01:50,0
820290,wYo3aBVj-bRPT7E4RSj5Kg,W5SNps2JaT_RozLAl_TN1Q,Worker there are the best at serving me quick ...,2017-12-17 01:10:52,0
873070,TEjijSowDwJM4vCL-zn4ew,YPHDzg1h-PkaxfoppT-iJg,bleed blue,2011-09-26 00:03:49,0


In [19]:
df = df.drop_duplicates(keep='first')
df

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11 19:18:57,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


In [20]:
nulos= df.isnull().sum()
nulos

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64